In [2]:
import sys
import time
import traceback
from arduino_iot_cloud import ArduinoCloudClient
import csv 
import plotly.io as pio
import plotly.express as px
import pandas as pd
import numpy as np
import dash
from dash import dcc, html, clientside_callback, ClientsideFunction, Patch
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import asyncio
import websockets
from dash.exceptions import PreventUpdate  # Import PreventUpdate
import orjson


In [3]:
# Define the variables to store the accelerometer data
cur_data = []
temp_data = []

In [4]:
# Initialize Dash app
app = dash.Dash(__name__)
plot_data = {
    'timestamp': [],
    'x': [],
    'y': [],
    'z': []
}
 
fig = go.Figure()
fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='X'))
fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Y'))
fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Z'))

app.layout = html.Div([
    dcc.Graph(figure = fig, id='live-update-graph'),
    dcc.Interval(id='interval-component', interval=100, n_intervals=0)  # Update every second  
])

@app.callback(
    Output('live-update-graph', 'figure'),
    Input('interval-component', 'n_intervals'),
    interval= 100,
)

def update_graph(n_intervals):
    global plot_data
    # Convert to DataFrame for Plotly
    patch = Patch()
    patch["data"] = [
        {"x": plot_data['timestamp'], "y": plot_data['x']},
        {"x": plot_data['timestamp'], "y": plot_data['y']},
        {"x": plot_data['timestamp'], "y": plot_data['z']}
    ]     # Replace Y values for Z trace
    return patch
app.run_server(debug =True)

In [8]:
def SmoothUpdate(dash_app = None, data = None, x_axis=None, y_axis=None, interval=25, store_id=None, interval_id=None, graph_id=None):
    """
    A function that creates smooth updates for a Dash graph using continuous data and client-side callbacks.
    The code stores real-time data in dcc.Store, and a client-side callback plots it, 
    using the first key for the x-axis and the rest for the y-axis, updating the graph.
    
    Args:
        dash_app: The Dash application instance.
        data: A dictionary or DataFrame with the data to be displayed.
        x_axis: The x-axis label/key in the data.
        y_axis: A list of y-axis labels/keys to plot.
        interval: The interval in milliseconds for updating the graph.
        store_id: The ID of the dcc.Store component that holds the data on the client-side.
        interval_id: The ID of the dcc.Interval component for time-based updates.
        graph_id: The ID of the dcc.Graph component to be updated.
        
    """ 
    variables = {
        "dash_app": dash_app,
        "data": data,
        "x_axis": x_axis,
        "y_axis": y_axis,
        "store_id": store_id,
        "interval_id": interval_id,
        "graph_id": graph_id
    }
    
    # Check if any variable is None and raise ValueError
    for var_name, var_value in variables.items():
        if var_value is None:
            raise ValueError(f"Error: '{var_name}' cannot be None.")        
    # Callback function to generate and update the data in the store component
    @dash_app.callback(
        Output(store_id, 'data'),  # This specifies the output to update the 'data' property of the dcc.Store component
        Input(interval_id, 'n_intervals'),  # This triggers the callback every time the interval component updates (based on the specified interval)
    )
    def generate_data(n_intervals):
        # Returns the updated plot data; in a real application, you would update this with new data
        return plot_data
    
    # Define the client-side callback to update the graph based on the store data
    dash_app.clientside_callback(
        """
        function(data) {
            const traces = [];  // Initialize an array to hold the graph traces
            const keys = Object.keys(data);  // Get the keys (columns) of the data object
            const x = keys[0];  // Assign the first key to the x-axis (e.g., 'timestamp')
    
            // Loop through each key (column) in the data
            for (let key in data) {
                if (key !== x) {  // Ensure the current key is not the x-axis key
                    traces.push({  // Push a new trace (line) for the graph with x and y values
                        x: data[x],  // X values from the 'x' column (e.g., 'timestamp')
                        y: data[key],  // Y values from the current column (e.g., 'x', 'y', 'z')
                        name: key  // Name the trace after the column (key)
                    });
                }
            }
    
            // Return the traces to update the graph data
            return {
                data: traces
            };
        }
        """,
        Output(graph_id, 'figure'),  # The output of the client-side callback updates the figure (graph) component
        Input(store_id, 'data')  # The callback is triggered whenever the data in the store component is updated
    )
    
# Initialize Dash app
app = dash.Dash(__name__)
plot_data = {
    'timestamp': [],
    'x': [],
    'y': [],
    'z': []
}
# Create initial empty DataFrame for the figure
df = pd.DataFrame(plot_data)
fig = px.line(df, x="timestamp", y=['x', 'y', 'z'])
# Define the app layout with graph, interval, and store components
app.layout = html.Div([
    dcc.Graph(figure=fig, id="live-update-graph"),
    dcc.Interval(id="interval-component", interval=25, n_intervals=0),
    dcc.Store(id="data-store")  # Store to hold data on client-side
])
plotter = SmoothUpdate(app, data=df, x_axis = 'timestamp', y_axis=['x', 'y', 'z'],store_id="data-store", interval_id="interval-component", graph_id="live-update-graph")
app.run_server(debug =True, port = 8051)

In [ ]:
DEVICE_ID = "89ea3640-6548-41ad-b0b4-ee33a5227bcd"
SECRET_KEY = "DHuvLL6j@8@OJqQFzKg3Y4mVP"

x, y, z = 0, 0, 0
count = 0
N_SAMPLES = 5

# =============================================================================
# Define the callback function for accelerometer_x changes
def on_accelerometer_x_changed(client, value):
    global x
    x = value

# Define the callback function for accelerometer_y changes
def on_accelerometer_y_changed(client, value):
    global y
    y = value

# Define the callback function for accelerometer_z changes
def on_accelerometer_z_changed(client, value):
    global z
    z = value

#================================= MAIN FUNCTION =================================
if __name__ == "__main__":
    try:
        # Instantiate Arduino cloud client
        client = ArduinoCloudClient(device_id=DEVICE_ID, username=DEVICE_ID, password=SECRET_KEY, sync_mode = True)
        
        # Register the callback functions
        client.register("py_x", value=None, on_write=on_accelerometer_x_changed)
        client.register("py_y", value=None, on_write=on_accelerometer_y_changed)
        client.register("py_z", value=None, on_write=on_accelerometer_z_changed)

        client.start()

        # Keep the client running
        while True:
            if x is not None and y is not None and z is not None:
                if count < N_SAMPLES:
                    count += 1
                    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
                    temp_data.append([count,timestamp,x,y,z])
                    #for key, data in zip(['timestamp', 'x', 'y', 'z'], [timestamp, x, y, z]):
                    #    plot_data[key].append(data)
                    print([count,timestamp,x,y,z])
                    x, y, z = None, None, None
                else:            
                    timestamp = time.strftime('%Y-%m-%d_%H-%M-%S')
                    df = pd.DataFrame(temp_data, columns=['index','timestamp', 'x', 'y', 'z'])
                    
                    for var in ['timestamp', 'x', 'y', 'z']:
                        plot_data[var] = df[var].to_numpy()
                    df.to_csv(f"csv_{timestamp}.csv")
                     
                    #fig = px.line(df, x= 'Timestamp', y = ['X','Y','Z'])  # Scatter plot
                    #filename = f"images/plot_{timestamp}.png"  # Correct filename format
                    #pio.write_image(fig, filename)

                    count = 0   
                    cur_data = temp_data.copy()
                    temp_data.clear()
            client.update()
    except:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        traceback.print_tb(exc_type, file=print)

[1, '2024-09-09 15:30:16', 0, 0, 0]
[2, '2024-09-09 15:30:16', 0.03460693359375, -0.9903106689453125, -0.120880126953125]
[3, '2024-09-09 15:30:17', 0.0355987548828125, -0.9901885986328125, -0.121826171875]
[4, '2024-09-09 15:30:18', 0.0368804931640625, -0.983734130859375, -0.1254425048828125]
[5, '2024-09-09 15:30:19', 0.03546142578125, -0.9925537109375, -0.11932373046875]
[1, '2024-09-09 15:30:20', 0.0383148193359375, -0.9901275634765625, -0.1250152587890625]
[2, '2024-09-09 15:30:21', 0.03851318359375, -0.99017333984375, -0.1221160888671875]
[3, '2024-09-09 15:30:22', 0.03729248046875, -0.9896087646484375, -0.1262664794921875]
[4, '2024-09-09 15:30:23', 0.0383758544921875, -0.9892120361328125, -0.123138427734375]
[5, '2024-09-09 15:30:24', 0.0359649658203125, -0.9897918701171875, -0.1230010986328125]
[1, '2024-09-09 15:30:25', 0.0369873046875, -0.990478515625, -0.121337890625]
[2, '2024-09-09 15:30:26', 0.03631591796875, -0.990509033203125, -0.1201934814453125]
[3, '2024-09-09 15:30